In [12]:
import os
import sys
import cv2
import csv
import tqdm
import pandas as pd
import random
import numpy as np
import time
from skimage.morphology import skeletonize, medial_axis
from matplotlib import pyplot as plt
from re import sub

In [13]:

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [28]:
CSV_SORT_PATH = "D:/DrugAge_thiot_GMC101/Sorted/604.csv"
df = pd.read_csv(CSV_SORT_PATH,names=('frame', 'x1', 'y1', 'x2', 'y2','label','delta'))
df['catagory'] = 'alive'
#df['X']=df[['x1','x2']].mean(axis=1)
#df['Y']=df[['y1','y2']].mean(axis=1)
#df = df[['frame', 'X', 'Y','label']]
uniqueFrames = df["frame"].unique()
uniqueIDs = df["label"].unique()

In [29]:
threshold = 150
vc = df.label.value_counts()
test = vc[vc > threshold].index.tolist()
csv_outputs = []
deadboxes = []
deathspots = []
fullboxes = []
for ID in test:        
    filtval = df['label'] ==ID
    interim = df[filtval]
    interimD = []
    interim2 = np.asarray(interim)
    fill = 0
    deadcount = 0
    for row in interim2:
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if fill > 1:
            boxA = [x1A, y1A, x2A, y2A]
            boxB = [x1B, y1B, x2B, y2B]
            deltaA = bb_intersection_over_union(boxA, boxB) 
            #print(deltaA)
            if deltaA > 0.95:
                deadcount += 1
                #print(deadcount)
            #print(fill)
            if deadcount > 15:
                catagoryA = 'dead'
               # print(deadcount)
            if deadcount == 16:
                if deadboxes == []:
                    deathspots.append([frameNA, x1A, y1A, x2A, y2A])     
                    deadboxes.append([x1A, y1A, x2A, y2A])  
                    csv_outputs.append((frameNA/144+1))
                else:
                    notunique = 0
                    for box in deadboxes:
                        #print(box)
                        x1D, y1D, x2D, y2D, *_ = box
                        boxD = [x1D, y1D, x2D, y2D]
                        deltaD = bb_intersection_over_union(boxA, boxD)  
                        if deltaD > 0.3:
                            notunique = 1
                    if notunique == 0:
                        deathspots.append([frameNA, x1A, y1A, x2A, y2A])  
                        deadboxes.append([x1A, y1A, x2A, y2A])               
                        csv_outputs.append((frameNA/144+1))

                #print(deathtime)
                #print(frameNA)
            #newRow = [frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA]
            #interimD.append(newRow)
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        fullboxes.append([frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA])  

        fill +=1
        #if deadcount == 16:
            #break
            
        #interimD.append(deathtime)
    #interimD = pd.DataFrame(interimD, columns = ['deathframe'])
    #csv_outputs.append(interimD)
#large_df = pd.concat(csv_outputs, ignore_index=True)  
csv_outputs = pd.DataFrame(csv_outputs, columns = ['#desc'])
csv_outputs['neural'] = '1'

#OUT_PATH = 'C:/Users/benja/Desktop/NN_alive_dead_training_vids/Sorted_Deathtimes/356_sorted_AD.csv'
#pd.DataFrame(csv_outputs).to_csv(OUT_PATH, mode='a', header=False, index=None)

In [30]:
deathspots2 = pd.DataFrame(fullboxes, columns = ['frameNA','x1', 'y1', 'x2', 'y2','label','delta','catagory'])
print(deathspots2)

      frameNA           x1          y1           x2          y2   label  \
0       155.0   845.732220  573.686738   861.293090  606.383425  1343.0   
1       156.0   845.413756  572.380266   861.779903  605.942127  1343.0   
2       157.0   845.308483  572.945610   861.900612  605.468141  1343.0   
3       158.0   844.842354  573.076202   860.917032  605.312217  1343.0   
4       159.0   845.374176  572.952314   861.258161  606.083338  1343.0   
...       ...          ...         ...          ...         ...     ...   
3565    525.0  1170.810864  552.263472  1189.893250  577.538859  1992.0   
3566    526.0  1170.181463  551.934770  1190.092039  578.020237  1992.0   
3567    527.0  1169.705134  551.975132  1189.096872  578.035509  1992.0   
3568    528.0  1169.341439  551.805506  1188.902676  577.570838  1992.0   
3569    529.0  1170.292181  551.326332  1189.702878  577.154750  1992.0   

         delta catagory  
0          NaN    alive  
1          NaN    alive  
2     0.957839    ali

In [23]:
pd.DataFrame(deathspots2).to_csv('C:/Users/benja/Desktop/831_tracked2.csv', mode='w', header=True, index=None)



In [33]:
VID_PATH = "C:/Users/benja/Downloads/604.avi"
vid = cv2.VideoCapture(VID_PATH)
OUT_PATH = "C:/Users/benja/Desktop"
total_frame_count = vid.get(cv2.CAP_PROP_FRAME_COUNT)
video_name = os.path.basename(VID_PATH).strip('.avi')
out_video_path = f"{OUT_PATH}/{os.path.basename(VID_PATH).strip('.avi')}_death4.avi"

while (1):
    ret, frame = vid.read()
    frame_count = vid.get(cv2.CAP_PROP_POS_FRAMES)
    #print(frame_count)
    if frame_count == 1:
        height, width, channels = frame.shape
        #print(height, width)
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(out_video_path, fourcc, 10, (width, height), True)
    
    for death in deathspots:
        #print(death)
        frameNA, x1, y1, x2, y2, *_ = death  
        frameNA = int(frameNA)
        if frame_count > frameNA:
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)
    
    writer.write(frame)
    if frame_count == total_frame_count:
        break
writer.release() 
print(out_video_path)

C:/Users/benja/Desktop/604_death4.avi


In [51]:
k = pd.DataFrame(['#desc','neural'])

In [70]:
print(csv_outputs)

       #desc neural
0     #expID    343
1   3.840278      1
2   5.284722      1
3   4.180556      1
4   5.881944      1
5   5.118056      1
6   3.111111      1
7   6.263889      1
8   6.354167      1
9   4.916667      1
10  5.513889      1
11  7.673611      1
12  4.506944      1
13  3.909722      1
14  6.423611      1
15  4.569444      1
16  6.680556      1
17  6.368056      1
18  5.208333      1
19  9.013889      1


In [69]:
csv_outputs.loc[0] = ['#expID','343']

In [66]:
#check  = pd.DataFrame(columns = column_names)
print(csv_outputs)

AttributeError: 'list' object has no attribute 'concat'

In [65]:
print(check)

[       #desc neural
0     #expID    343
1     #expID    343
2   5.284722      1
3   4.180556      1
4   5.881944      1
5   5.118056      1
6   3.111111      1
7   6.263889      1
8   6.354167      1
9   4.916667      1
10  5.513889      1
11  7.673611      1
12  4.506944      1
13  3.909722      1
14  6.423611      1
15  4.569444      1
16  6.680556      1
17  6.368056      1
18  5.208333      1
19  9.013889      1,        #desc neural
0     #expID    343
1     #expID    343
2   5.284722      1
3   4.180556      1
4   5.881944      1
5   5.118056      1
6   3.111111      1
7   6.263889      1
8   6.354167      1
9   4.916667      1
10  5.513889      1
11  7.673611      1
12  4.506944      1
13  3.909722      1
14  6.423611      1
15  4.569444      1
16  6.680556      1
17  6.368056      1
18  5.208333      1
19  9.013889      1]


In [75]:
csv_list = os.listdir('D:/NN_alive_dead_training_vids/Sorted_Deathtimes_Thresh')
out_csv_path = 'C:/Users/benja/Desktop/AllN2.csv'
count = 0
for csv_name in csv_list:
    csv_path = os.path.join('D:/NN_alive_dead_training_vids/Sorted_Deathtimes_Thresh', csv_name)
    df = pd.read_csv(csv_path)
    if count == 0:
        pd.DataFrame(df).to_csv(out_csv_path, mode='w', header=True, index=None)
    else:
        pd.DataFrame(df).to_csv(out_csv_path, mode='a', header=True, index=None)
    count+=1    

                 #desc  neural
0               #expID     343
1    10.63888888888889       1
2    11.26388888888889       1
3   11.666666666666666       1
4   11.930555555555555       1
5                13.75       1
6    9.458333333333332       1
7    12.38888888888889       1
8    9.708333333333332       1
9   11.333333333333334       1
10  14.430555555555555       1
11               #desc  manual
12              #expID     343
13                  14       1
14                  13       1
15                  13       1
16                  11       1
17                  11       1
18                  10       1
19                  10       1
20                  11       1
21                  14       1


In [5]:
vid = cv2.VideoCapture('E:/1217/day6.avi')
total_frame_count = vid.get(cv2.CAP_PROP_FRAME_COUNT)
print(total_frame_count)

9141.0
